In [1]:
from models.PCA import PCA
from models.FF import FF

import pandas as pd
import numpy as np
from tqdm import tqdm
from utils import charas
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def model_inference_and_predict(model):
    mon_list = pd.read_pickle('data/mon_list.pkl')
    test_mons = mon_list.loc[mon_list >= model.test_period[0]]
    inference_result = []
    predict_result = []
    T_bar = tqdm(test_mons.groupby(test_mons.apply(lambda x: x//10000)), colour='red', desc=f'{model.name} Inferencing & Predicting')
    
    for g in T_bar: # rolling train
        T_bar.set_postfix({'Year': g[0]})
        model.train_model()
        
        for m in g[1].to_list():
            inference_result.append(model.inference(m))
            predict_result.append(model.predict(m))
        # model refit (change train period and valid period)
        model.refit()

    inference_result = pd.DataFrame(inference_result, index=test_mons, columns=charas)
    inference_result.to_csv(f'results/inference/{model.name}_inference.csv')
    
    predict_result = pd.DataFrame(predict_result, index=test_mons, columns=charas)
    predict_result.to_csv(f'results/predict/{model.name}_predict.csv')

In [ ]:
pca_1 = PCA(K=1, portfolio=True)
pca_2 = PCA(K=2, portfolio=True)
pca_3 = PCA(K=3, portfolio=True)
pca_4 = PCA(K=4, portfolio=True)
pca_5 = PCA(K=5, portfolio=True)
pca_6 = PCA(K=6, portfolio=True)

In [ ]:
model_inference_and_predict(pca_1)
# model_inference_and_predict(pca_2)
# model_inference_and_predict(pca_3)
# model_inference_and_predict(pca_4)
# model_inference_and_predict(pca_5)
# model_inference_and_predict(pca_6)

In [ ]:
ff_1 = FF(K=1, portfolio=True)
ff_2 = FF(K=2, portfolio=True)
ff_3 = FF(K=3, portfolio=True)
ff_4 = FF(K=4, portfolio=True)
ff_5 = FF(K=5, portfolio=True)
ff_6 = FF(K=6, portfolio=True)

In [2]:
model_inference_and_predict(ff_1)
model_inference_and_predict(ff_2)
model_inference_and_predict(ff_3)
model_inference_and_predict(ff_4)
model_inference_and_predict(ff_5)
model_inference_and_predict(ff_6)


KeyboardInterrupt



In [12]:
def model_inference_and_predict_CA(model):
    mon_list = pd.read_pickle('data/mon_list.pkl')
    test_mons = mon_list.loc[mon_list >= model.test_period[0]]
    inference_result = pd.DataFrame()
    predict_result = pd.DataFrame()
    T_bar = tqdm(test_mons.groupby(test_mons.apply(lambda x: x//10000)), colour='red', desc=f'{model.name} Inferencing & Predicting')
    
    stock_index = pd.Series(dtype=np.int64)
    for g in T_bar: # rolling train
        T_bar.set_postfix({'Year': g[0]})
        model.train_model()
        
        for m in g[1].to_list():
            m_stock_index, _, _, _ = model._get_item(m)
            stock_index = pd.concat([stock_index, pd.Series(m_stock_index)]).drop_duplicates().astype(int)
            inference_R = model.inference(m) # return (N, 1)
            predict_R = model.inference(m) # reutrn (N, 1)

            # move inference_R and predict_R to cpu
            inference_R = inference_R.cpu().detach().numpy()
            predict_R = predict_R.cpu().detach().numpy()

            inference_R = pd.DataFrame(inference_R, index=m_stock_index, columns=[m])
            predict_R = pd.DataFrame(predict_R, index=m_stock_index, columns=[m])

            
            inference_result = pd.concat([inference_result, inference_R], axis=1) # (N, T)
            predict_result = pd.concat([predict_result, predict_R], axis=1) # (N, T)

            # DEBUG:
            # save inference_R and predict_R to csv
            inference_R.to_csv(f'temp/{model.name}_inference_stock_{m}.csv')
            predict_R.to_csv(f'temp/{model.name}_predict_stock_{m}.csv')
            
        # model refit (change train period and valid period)
        model.refit()

    inference_result = pd.DataFrame(inference_result, index=test_mons, columns=charas)
    inference_result.to_csv(f'results/inference/{model.name}_inference_stock.csv')
    
    predict_result = pd.DataFrame(predict_result, index=test_mons, columns=charas)
    predict_result.to_csv(f'results/predict/{model.name}_predict_stock.csv')
    return inference_result, predict_result

In [13]:
from models.CA import CA0, CA1, CA2, CA3
import gc

In [14]:
ca3_lists = []
for i in range(6):
    gc.collect()
    ca3_lists.append(CA3(i + 1).to('cuda'))
    print(f'begin of {ca3_lists[i].name}')
    model_inference_and_predict_CA(ca3_lists[i])


begin of CA3_1


CA3_1 Inferencing & Predicting:   0%|          | 0/30 [00:00<?, ?it/s, Year=1987]

Index([33652, 33653, 33654, 33655, 33656, 33657, 33658, 33659, 33660, 33661,
       33662, 33663, 33664, 33665, 33666, 33667, 33668, 33669, 33670, 33671,
       33672, 33673, 33674, 33675, 33676, 33677, 33678, 33679, 33680, 33681,
       33682, 33683, 33684, 33685, 33686, 33687, 33688, 33689, 33690, 33691,
       33692, 33693, 33694, 33695, 33696, 33697, 33698, 33699, 33700, 33701,
       33702, 33703, 33704, 33705, 33706, 33707, 33708, 33709, 33710, 33711,
       33712, 33713, 33714, 33715, 33716, 33717, 33718, 33719, 33720, 33721,
       33722, 33723, 33724, 33725, 33726, 33727, 33728, 33729, 33730, 33731,
       33732, 33733, 33734, 33735, 33736, 33737, 33738, 33739, 33740, 33741,
       33742, 33743, 33744, 33745],
      dtype='int64')
Index([33746, 33747, 33748, 33749, 33750, 33751, 33752, 33753, 33754, 33755,
       33756, 33757, 33758, 33759, 33760, 33761, 33762, 33763, 33764, 33765,
       33766, 33767, 33768, 33769, 33770, 33771, 33772, 33773, 33774, 33775,
       33776, 33777

CA3_1 Inferencing & Predicting:   3%|▎         | 1/30 [00:16<08:00, 16.56s/it, Year=1988]

Index([34780, 34781, 34782, 34783, 34784, 34785, 34786, 34787, 34788, 34789,
       34790, 34791, 34792, 34793, 34794, 34795, 34796, 34797, 34798, 34799,
       34800, 34801, 34802, 34803, 34804, 34805, 34806, 34807, 34808, 34809,
       34810, 34811, 34812, 34813, 34814, 34815, 34816, 34817, 34818, 34819,
       34820, 34821, 34822, 34823, 34824, 34825, 34826, 34827, 34828, 34829,
       34830, 34831, 34832, 34833, 34834, 34835, 34836, 34837, 34838, 34839,
       34840, 34841, 34842, 34843, 34844, 34845, 34846, 34847, 34848, 34849,
       34850, 34851, 34852, 34853, 34854, 34855, 34856, 34857, 34858, 34859,
       34860, 34861, 34862, 34863, 34864, 34865, 34866, 34867, 34868, 34869,
       34870, 34871, 34872, 34873],
      dtype='int64')
Index([34874, 34875, 34876, 34877, 34878, 34879, 34880, 34881, 34882, 34883,
       34884, 34885, 34886, 34887, 34888, 34889, 34890, 34891, 34892, 34893,
       34894, 34895, 34896, 34897, 34898, 34899, 34900, 34901, 34902, 34903,
       34904, 34905

...

CA3_1 Inferencing & Predicting: 100%|██████████| 30/30 [12:33<00:00, 25.11s/it, Year=2016]


begin of CA3_2


CA3_2 Inferencing & Predicting:   0%|          | 0/30 [00:16<?, ?it/s, Year=1987]

Index([33652, 33653, 33654, 33655, 33656, 33657, 33658, 33659, 33660, 33661,
       33662, 33663, 33664, 33665, 33666, 33667, 33668, 33669, 33670, 33671,
       33672, 33673, 33674, 33675, 33676, 33677, 33678, 33679, 33680, 33681,
       33682, 33683, 33684, 33685, 33686, 33687, 33688, 33689, 33690, 33691,
       33692, 33693, 33694, 33695, 33696, 33697, 33698, 33699, 33700, 33701,
       33702, 33703, 33704, 33705, 33706, 33707, 33708, 33709, 33710, 33711,
       33712, 33713, 33714, 33715, 33716, 33717, 33718, 33719, 33720, 33721,
       33722, 33723, 33724, 33725, 33726, 33727, 33728, 33729, 33730, 33731,
       33732, 33733, 33734, 33735, 33736, 33737, 33738, 33739, 33740, 33741,
       33742, 33743, 33744, 33745],
      dtype='int64')


AssertionError: Dimension mismatch between mon_factor: torch.Size([1, 2]) and mon_beta: torch.Size([94, 2])